In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.012973785400390625
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 28.70729289407867
200000 46.22946975801675
300000 57.721194057917174
400000 65.33165107717038
500000 73.39157337091936
600000 73.13496461331043
700000 69.55765204211994
800000 69.2314308020962
900000 69.09580663504009
1000000 69.09811532474049
1100000 21.325750478146084
1200000 70.63939550949806
1300000 68.41434823290274
1400000 66.91544483328
1500000 67.0822352031279
1600000 70.6092611715605
1700000 68.373678622323
1800000 67.37483614489994
1900000 67.68127140232188
2000000 68.30260636419557
2100000 53.372526995247796
2200000 36.69118103452964
2300000 66.44118895309693
2400000 65.73379317757198
2500000 65.92531092262092
2600000 65.777107916124
2700000 68.93137079798683
2800000 65.14533112781761
2900000 65.93303463492802
3000000 61.95590007592251
3100000

30100000 67.96660889834536
30200000 68.22304252605855
30300000 65.14267765154626
30400000 56.67914187959456
30500000 37.934981702609214
30600000 68.33516652475782
30700000 67.09781953527792
30800000 71.08860256500384
30900000 68.6979016514486
31000000 73.28911161231808
31100000 72.80218580661432
31200000 75.90004658776401
31300000 35.136107306316674
31400000 75.28306099202466
31500000 74.53060545240245
31600000 83.34255010726521
31700000 59.77245872881138
31800000 45.12766376484773
31900000 77.68929832544268
32000000 75.52703109437887
32100000 72.98704039685568
32200000 70.80834340890257
32300000 70.89877031240253
32400000 66.04821532977417
32500000 52.778618588792504
32600000 43.1408617487593
32700000 71.28074454809331
32800000 68.28836880718559
32900000 72.79228963279085
33000000 71.63723158127827
33100000 70.97840616395469
33200000 73.23216476795072
33300000 63.0242026251933
33400000 46.87420632925794
33500000 74.79724251064546
33600000 75.70578667937706
33700000 76.99732028616727
3

60500000 67.01565599722998
60600000 70.59163967394257
60700000 68.95584115744
60800000 76.10188497539379
60900000 73.81013857101948
61000000 74.55195231484839
61100000 71.62335852697683
61200000 76.6226880660482
61300000 50.24835319700896
61400000 51.39734697564107
61500000 66.91276091220016
61600000 63.81878198474958
61700000 71.86791530606341
61800000 66.91563415246905
61900000 32.036719533150794
62000000 63.08501465760623
62100000 67.07819914492383
62200000 66.80015004909986
62300000 68.3840244762079
62400000 70.18050184844812
62500000 30.079561882040558
62600000 71.41196822411526
62700000 72.6447466395236
62800000 73.68471216160789
62900000 76.60388522712263
63000000 74.95293059972812
63100000 71.70243999520638
63200000 72.47180807619776
63300000 71.03509538451826
63400000 59.32557219855254
63500000 42.68090434160911
63600000 63.4009102900802
63700000 66.22834696984481
63800000 66.43677081645617
63900000 63.981855011369774
64000000 29.8760351897449
64100000 60.58184101884328
642000

90800000 64.58030849026227
90900000 41.45296786818455
91000000 48.58531671255256
91100000 65.3777839110414
91200000 58.932273415120996
91300000 23.551542303102543
91400000 64.68014787351903
91500000 62.68208011451944
91600000 38.143972861471894
91700000 58.37689484078147
91800000 62.79928686486095
91900000 65.21890612656497
92000000 64.84243275343208
92100000 67.96170497655724
92200000 67.23359390335834
92300000 70.45305298429612
92400000 64.87955042096009
92500000 69.43998079956745
92600000 67.05502302874021
92700000 66.81770515970432
92800000 63.689901923237926
92900000 67.25942293475957
93000000 46.43379658975955
93100000 44.43583520228537
93200000 59.90835207434372
93300000 58.19412333466614
93400000 26.434563478327128
93500000 60.26970128982607
93600000 59.61559721171418
93700000 29.60383220723382
93800000 66.29984668126822
93900000 60.567728409335416
94000000 59.81701134083272
94100000 70.93639860954318
94200000 68.57038119705464
94300000 68.70985065766568
94400000 68.72609568050

120300000 59.41690697135814
120400000 55.86634506059539
120500000 35.975139135840834
120600000 28.925844281697433
120700000 32.846220360183196
120800000 42.800282867412
120900000 62.54114300655693
121000000 67.68244274482518
121100000 63.196424653074786
121200000 61.84820784702424
121300000 64.12756997436719
121400000 61.36054854317269
121500000 68.43856251941838
121600000 67.56057336769962
121700000 65.87408131547657
121800000 59.49811983076678
121900000 62.14165414692684
122000000 60.97071734977967
122100000 58.76024539605529
122200000 59.466322970320874
122300000 61.35503555014263
122400000 56.670117649041956
122500000 59.65043816675636
122600000 46.64090554298008
122700000 22.281470828770892
122800000 24.20849216339688
122900000 53.52756680774126
123000000 56.5311059410113
123100000 57.46011882973511
123200000 60.80071171759216
123300000 62.04226216944166
123400000 56.69186962008708
123500000 60.02820836252142
123600000 58.612652952886556
123700000 65.10693831309005
123800000 67.47

...

799200000 0.2492688600645655
799300000 0.5239356095266382
best so far: 0
type: [1, 2, 8, 2, 4] 128
cases of this type: 262144
799400000 0.5595084706729173
799500000 0.24870627383685312
best so far: 0
type: [1, 2, 8, 4, 2] 128
cases of this type: 131072
799600000 0.24462699029653076
799700000 0.2205485452736795
best so far: 0
type: [1, 2, 8, 8, 1] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 1, 4] 128
cases of this type: 131072
799800000 0.2291568493182381
799900000 0.24661716730731129
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.22114392235835592
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.4737317103979018
800200000 1.3008733971717

812600000 1.1479393615738922
812700000 1.0395920900969937
812800000 1.0965272663442809
812900000 1.051712840516276
813000000 0.9513678411408373
813100000 0.8643719554467261
813200000 0.9422551588286605
813300000 0.9920889248783662
813400000 0.887957450112025
813500000 0.8263827871435899
813600000 0.784094713475883
813700000 0.8515001153115392
813800000 0.9850504353355679
813900000 0.8329466762098828
814000000 0.8207795517428756
814100000 0.8846099428518587
814200000 0.910080820236063
814300000 0.8137445956416209
814400000 0.7934284521215824
814500000 0.8125950646013902
814600000 0.8863982730753421
814700000 0.9043614971617526
814800000 0.9422629638260623
814900000 0.913391782224387
815000000 0.7755633991020674
815100000 0.779439936959331
815200000 0.9402657841020565
815300000 0.7700159904404673
815400000 0.9406814277093286
815500000 0.8249583804419061
815600000 0.7385698920725048
815700000 0.8718612054076811
815800000 0.7184869621033847
815900000 0.8864934727964833
816000000 1.11007049

840300000 0.22889842645544276
840400000 0.22849881493781407
840500000 0.15198932376945284
840600000 0.23381462204906145
840700000 0.17228666932627162
840800000 0.18839527841631504
840900000 0.22684497253137365
841000000 0.1298233477908969
841100000 0.23731193152351315
841200000 0.2343048185564326
841300000 0.15956571897449692
841400000 0.22209087142873477
841500000 0.21511351823708283
841600000 0.22230735461519557
841700000 0.23269590223907963
841800000 0.14095518887086708
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.25143319698624017
842000000 0.18225593675238888
842100000 0.14767039320889577
842200000 0.22211281466181276
842300000 0.1531826459335069
842400000 0.1759963855797695
842500000 0.1951734545721213
842600000 0.13811847695072957
842700000 0.199708305974004
842800000 0.15687670065739154
842900000 0.14958924175197613
843000000 0.19141389270473189
843100000 0.13479557693826158
843200000 0.17431307368480695
843300000 0.17072384378471575
84340000

best so far: 0
type: [2, 2, 1, 32, 1] 128
cases of this type: 65536
860800000 0.015050208537495136
best so far: 0
type: [2, 2, 2, 1, 16] 128
cases of this type: 524288
860900000 0.028301772202531498
861000000 0.03617274629404744
861100000 0.024763546758431197
861200000 0.016350415602126387
861300000 0.033440810719350976
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.03275532336485386
861500000 0.02728586043379704
861600000 0.014657266114016375
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.024408984945732355
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.018371462729673254
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.018790571843312183
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.017017201221891244
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this ty

866100000 0.0005888368220918708
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.0001920635153591633
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so f